In [1]:
import sys
import os
import math
import moveit_commander
import moveit_msgs.msg
import rospy
import numpy as np
import trajectory_msgs.msg
from geometry_msgs.msg import PoseStamped
import tf
import actionlib
import subprocess

### ALL OF THESE IMPORTS MUST BE HANDLeD BY TAKESHI INSTALL

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


In [2]:
rospy.init_node('moveiteando')

In [3]:
listener = tf.TransformListener()

In [4]:
arm = moveit_commander.MoveGroupCommander('arm')
base=  moveit_commander.MoveGroupCommander('base')
gripper = moveit_commander.MoveGroupCommander('gripper')
head = moveit_commander.MoveGroupCommander('head')

In [5]:
def delete_object(name):
    u"""Gazeboの物体を消す関数

    引数:
        name (str): 物体の名前

    """

    cmd = ['rosservice', 'call', 'gazebo/delete_model',
           '{model_name: ' + str(name) + '}']
    subprocess.call(cmd)

    



In [6]:
#####SPAWN OBJECTS
from gazebo_ros import gazebo_interface
from geometry_msgs.msg import Pose, Quaternion
def spawn_object(gazebo_name, name, x, y, z, yaw):
    rospy.loginfo('Spawn: {0}'.format(name))
    initial_pose = Pose()
    initial_pose.position.x = x
    initial_pose.position.y = y
    initial_pose.position.z = z
    roll = 0.0
    pitch = 0.0
    yaw = 0.0
    q = tf.transformations.quaternion_from_euler(roll, pitch, yaw)
    initial_pose.orientation = Quaternion(q[0], q[1], q[2], q[3])

    path_xml = _path_xml.replace('MODEL_NAME', name)

    with open(path_xml, "r") as f:
        model_xml = f.read()

    model_xml = model_xml.replace('PATH_TO_MODEL', _path_model)

    gazebo_interface.spawn_sdf_model_client(gazebo_name, model_xml, rospy.get_namespace(),
                                            initial_pose, "", "/gazebo")


In [7]:
## THIS PATH MUST BE SET BY USER ()

_path_xml = "/home/roboworks/Codes/ycb_ws/src/robot_object_views/robotobject/models/MODEL_NAME/model-1_4.sdf"
_path_model = "/home/roboworks/Codes/ycb_ws/src/robot_object_views/robotobject/models"

model_database_template = """<sdf version="1.4">
  <world name="default">
    <include>
      <uri>model://MODEL_NAME</uri>
    </include>
  </world>
</sdf>"""

objs=os.listdir(_path_model)
print (objs)

['004_sugar_box', '007_tuna_fish_can', '017_orange', '065-h_cups', '070-b_colored_wood_blocks', '077_rubiks_cube', '070-a_colored_wood_blocks', '054_softball', '073-c_lego_duplo', '011_banana', '013_apple', '037_scissors', '065-d_cups', '050_medium_clamp', '012_strawberry', '065-j_cups', '031_spoon', '072-c_toy_airplane', '061_foam_brick', '018_plum', '008_pudding_box', '072-e_toy_airplane', '065-e_cups', '030_fork', '065-b_cups', '056_tennis_ball', '022_windex_bottle', '042_adjustable_wrench', '059_chain', '006_mustard_bottle', '065-i_cups', '073-b_lego_duplo', '072-a_toy_airplane', '038_padlock', '072-b_toy_airplane', '019_pitcher_base', '043_phillips_screwdriver', '052_extra_large_clamp', '073-g_lego_duplo', '002_master_chef_can', '010_potted_meat_can', '029_plate', '051_large_clamp', '040_large_marker', '065-c_cups', '063-a_marbles', '036_wood_block', '033_spatula', '072-d_toy_airplane', '016_pear', '026_sponge', '003_cracker_box', '048_hammer', '063-b_marbles', '015_peach', '065-g

In [8]:
def gazebo_2_world(x,y):

    x_world= x+2.0
    y_world= y-1.0
    return (x_world,y_world)

def world_2_gazebo(x_world,y_world):

    x= ( x_world - 2.5)
    y= (-y_world + 1.5) 
    return (x , y)

In [9]:
z= 0 ## QUERY given Z where does the gaze ponits to wrt map


(trans,rot) = listener.lookupTransform('/map', '/head_rgbd_sensor_rgb_frame', rospy.Time(0))
e = np.rad2deg(tf.transformations.euler_from_quaternion(rot))
D_map=np.abs(trans[2]-z)/np.tan(np.deg2rad(e[0]-(-90)))
print ('x,y',D_map*-1* np.sin(np.deg2rad(e[2])), D_map* np.cos(np.deg2rad(e[2]))  )



('x,y', 1091.3911264536378, -5.065504497537456)


In [10]:
model_name= "073-g_lego_duplo"




x_gaz,y_gaz=world_2_gazebo(1.2,1)


spawn_object(model_name, model_name, x_gaz,  y_gaz, .21, 0)  ### ORigin in gazebo world is in (-2, 1 ) map coords
#print (x_gaz,y_gaz)

#spawn_object("073-g_lego_duplo", "073-g_lego_duplo", x_gaz, y_gaz, .51, 0)



[INFO] [1610125720.293769, 178.143000]: Spawn: 073-g_lego_duplo
[INFO] [1610125720.296262, 178.143000]: Waiting for service /gazebo/spawn_sdf_model
[INFO] [1610125720.303502, 178.143000]: Calling service /gazebo/spawn_sdf_model
[INFO] [1610125720.380390, 178.162000]: Spawn status: SpawnModel: Successfully spawned entity


In [20]:
def gaze_point_floor(x,y):
    
    arm_pose=arm.get_current_joint_values()
    arm_pose[0]=.1
    arm_pose[1]= -0.3
    arm.set_joint_value_target(arm_pose)
    arm.go()
    head_pose = head.get_current_joint_values()
    head_pose[0]=0.0
    head_pose[1]=0.0
    head.set_joint_value_target(head_pose)
    head.go()
    trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0)) #
    np.rad2deg(head_pose)
    e =tf.transformations.euler_from_quaternion(rot)
    print('i am at',trans[:2],np.rad2deg(e)[2])
    print('gaze goal',x,y)
    #tf.transformations.euler_from_quaternion(rot)


    x_rob,y_rob,th_rob= trans[0], trans[1] ,  e[2]


    D_x=x_rob-x
    D_y=y_rob-y
    D_th= np.arctan2(D_y,D_x)
    print('relative to robot',(D_x,D_y,np.rad2deg(D_th)))

    pan_correct= (- th_rob + D_th + np.pi) % (2*np.pi)

    if(pan_correct > np.pi):
        pan_correct=-2*np.pi+pan_correct
    if(pan_correct < -np.pi):
        pan_correct=2*np.pi+pan_correct

    if ((pan_correct) > .5 * np.pi):
        print ('Exorcist alert')
        pan_correct=.5*np.pi
    head_pose[0]=pan_correct
    tilt_correct=np.arctan2(trans[2],np.linalg.norm((D_x,D_y)))

    head_pose [1]=-tilt_correct
    
    
    
    head.set_joint_value_target(head_pose)
    head.go()

In [21]:
gaze_point_floor(1,1)

('i am at', [-0.05889733901018696, 0.02140851360943641], -0.6597231147450632)
('gaze goal', 1, 1)
('relative to robot', (-1.058897339010187, -0.9785914863905636, -137.257095096097))


In [33]:
np.linalg.norm((-1.0601638321311557, -0.9780615410155707))


1.442411775108887

In [34]:
trans , rot = listener.lookupTransform('/map', '/head_rgbd_sensor_gazebo_frame', rospy.Time(0)) #
D_x,D_y = -1.0596602015879786, -0.9782633283138407
np.linalg.norm((D_x,D_y))



1.4421785195852657

In [35]:
tilt_correct=np.arctan2(trans[2],np.linalg.norm((D_x,D_y)))

In [76]:
if(pan_correct > np.pi):
    print (pan_correct)
    print ('better',-2*np.pi+pan_correct)
if(pan_correct < -np.pi):
    print (pan_correct)
    print ('better',2*np.pi+pan_correct)
    head_pose[0]=2*np.pi+pan_correct
    if (2*np.pi+pan_correct >np.pi*.5):
        print ('exorcist alert')
        head_pose[0]=np.pi*.5
else:
    head_pose[0]=pan_correct
head.set_joint_value_target(head_pose)
head.go()

True

In [37]:
head_pose = head.get_current_joint_values()
head_pose[1]= -tilt_correct
head.set_joint_value_target(head_pose)
head.go()

True

In [29]:
tilt_correct

NameError: name 'tilt_correct' is not defined

In [30]:
head_pose = head.get_current_joint_values()


NameError: name 'tilt_correct' is not defined

In [14]:
arm.get_active_joints(),arm.get_current_joint_values()


(['arm_lift_joint',
  'arm_flex_joint',
  'arm_roll_joint',
  'wrist_flex_joint',
  'wrist_roll_joint',
  'wrist_ft_sensor_frame_joint'],
 [0.0913309879955888,
  -2.5024671537207652,
  -1.5692579000708107,
  -1.5701602929505043,
  -3.813824795440013e-05,
  0.0])

True

In [41]:
head_pose = head.get_current_joint_values()
head_pose[0]=0.0
head_pose[1]=0.0
head.set_joint_value_target(head_pose)
head.go()

True

In [38]:
tilt_correct

0.6025695789184932

AttributeError: 'float' object has no attribute 'np'